In [1]:
import sys
sys.path.append('..')
from run.tools import *

/home/xiangyan/anaconda3/envs/graphdot/lib/python3.7/site-packages/tqdm/std.py:706: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# setup
optimizer = None
kernel, alpha = set_kernel_alpha('graph:0.01')
single_graph, multi_graph, reaction_graph, properties = \
    set_graph_property('SMILES:::tc')
add_f, add_p = set_add_feature_hyperparameters(None)
kernel_config = set_kernel_config(
    kernel, add_f, add_p,
    single_graph, multi_graph, '../hyperparameters/hyper3.json',
    'tc',
)
df = get_df('../run/datasets/critical-sim.txt', 'tc.pkl', single_graph, multi_graph, [])

/home/xiangyan/anaconda3/envs/graphdot/lib/python3.7/site-packages/tqdm/std.py:706: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Transforming molecules into graphs. (pure compounds)


100%|██████████| 3617/3617 [00:36<00:00, 98.95it/s] 


In [3]:
# randomly select 100 molecules to train the model as a example.
df = df.sample(100)
X, y = df[['SMILES']].to_numpy(), df['tc'].to_numpy()

In [4]:
# GPR using sklearn
from chemml.regression.GPRsklearn.gpr import GPR
gpr = GPR(kernel=kernel_config.kernel, optimizer=optimizer, alpha=alpha,y_scale=True)
gpr.fit(X, y)

GPR(alpha=0.01,
    kernel=<chemml.kernels.GraphKernel.NormalizationMolSize object at 0x7f0fb1cb0450>,
    optimizer=None)

In [5]:
# prediction
smiles='CCCCCCO'
X_ = np.asarray([HashGraph.from_smiles(smiles, rdkit_config(), smiles)])
unify_datatype(X.ravel(), X_)
gpr.predict(X_, return_std=True)
# The uncertainty of sklearn GPR is normalized to [0,1]

(array([663.90003954]), array([0.30813588]))

In [6]:
# GPR using graphdot
from chemml.regression.GPRgraphdot.gpr import GPR
gpr = GPR(kernel=kernel_config.kernel, optimizer=optimizer, alpha=alpha, normalize_y=True)
gpr.fit(X, y)

In [7]:
# prediction
gpr.predict(X_, return_std=True)

(array([663.89994422]), array([0.30813596]))